Pada Soal Bonus ini saya mengerjakan technical challenges yang ketiga yaitu:  **Compare 3 configurations for the activation function. Show and
explain your performance result.**


#Import Library

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets

import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score


#Build Function


Untuk menghindari penulisan model yang berulang-ulang, maka untuk bagian neural network design, Set hyperparameter, training serta model evaluation akan dimasukkan kedalam sebuah function.

In [ ]:
def build_model_nn(activation):

  #Load Dataset MNIST dari Pytorch
  transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

  train_data = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
  test_data = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

  train_loader = torch.utils.data.DataLoader(train_data, batch_size=100, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

  #Mendesain Neural Network
  class Model(nn.Module):
    def __init__(self):
      super(Model, self).__init__()
      self.encoder1 = nn.Linear(28*28, 256)
      self.encoder2 = nn.Linear(256, 128)
      self.encoder3 = nn.Linear(128, 64)
      self.encoder4 = nn.Linear(64, 32)
      self.encoder5 = nn.Linear(32, 10)
      self.activation = activation #Agar model dapat secara otomatis menggunakan activation function yang diinput ke function

    def forward(self, x):
      x = x.view(-1, 28 * 28)
      x = self.activation(self.encoder1(x))
      x = self.activation(self.encoder2(x))
      x = self.activation(self.encoder3(x))
      x = self.activation(self.encoder4(x))
      x = self.encoder5(x)
      return x
  model = Model()

  #Set Hyperparameter
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

  #Training Loop MNIST DataSets
  start_time = time.time()
  losses = []
  print(f'Training MNIST Datasets using {activation} activation')
  for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i%200 == 0:
          print(f'Epoch : {epoch}, batch : {i}, loss : {loss.item()}')
        losses.append(loss.detach().numpy())
  total_time = time.time() - start_time
  print(f'Finished Training using ({activation}) activation, in Durattion: {total_time/60} minutes.')
  print(f'====================================================================================================')
  print('')
  print(f'====================================================================================================')

  #Model Evaluation
  model.eval()
  correct = 0
  total = 0
  predicted_labels = []
  true_labels = []

  with torch.no_grad():
      for data in test_loader:
          images, labels = data
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

          predicted_labels.extend(predicted.numpy())
          true_labels.extend(labels.numpy())

  accuracy = accuracy_score(true_labels, predicted_labels)
  f1 = f1_score(true_labels, predicted_labels, average='weighted')
  precision = precision_score(true_labels, predicted_labels, average='weighted')
  recall = recall_score(true_labels, predicted_labels, average='weighted')
  confusion = confusion_matrix(true_labels, predicted_labels)

  return {
      'activation': activation,
      'accuracy': accuracy,
      'f1_score': f1,
      'precision': precision,
      'recall': recall,
      'confusion_matrix': confusion,
      'duration': total_time
  }

#Menjalankan Fungsi

In [ ]:
#membuat variabel yang menyimpan fungsi aktivasi yang akan digunakan.
#disini ada 3 fungsi aktivasi yang digunakan yaitu sigmoid, Tanh dan ReLU
activation_functions = [nn.Sigmoid(), nn.Tanh(), nn.ReLU()]

model_performances = [] #menyimpan hasil run fungsi yang telah dibuat sebelumnya


#menjalankan fungsi build_model_nn dan menyimpan outputnya ke variabel model_performances
for activation_function in activation_functions:
  result = build_model_nn(activation_function)
  model_performances.append(result)

Training MNIST Datasets using Sigmoid() activation
Epoch : 0, batch : 0, loss : 2.3310723304748535
Epoch : 0, batch : 200, loss : 1.4959242343902588
Epoch : 0, batch : 400, loss : 0.8760689496994019
Epoch : 1, batch : 0, loss : 0.5416803956031799
Epoch : 1, batch : 200, loss : 0.3893685042858124
Epoch : 1, batch : 400, loss : 0.29255473613739014
Epoch : 2, batch : 0, loss : 0.29812610149383545
Epoch : 2, batch : 200, loss : 0.3325211703777313
Epoch : 2, batch : 400, loss : 0.18727390468120575
Epoch : 3, batch : 0, loss : 0.19136124849319458
Epoch : 3, batch : 200, loss : 0.20256055891513824
Epoch : 3, batch : 400, loss : 0.15395168960094452
Epoch : 4, batch : 0, loss : 0.09814225882291794
Epoch : 4, batch : 200, loss : 0.16047781705856323
Epoch : 4, batch : 400, loss : 0.18687711656093597
Epoch : 5, batch : 0, loss : 0.06021694839000702
Epoch : 5, batch : 200, loss : 0.1251239776611328
Epoch : 5, batch : 400, loss : 0.2283354252576828
Epoch : 6, batch : 0, loss : 0.048351746052503586
E

In [ ]:
#Melakukan print hasil dari performance model yang telah di jalankan
for result in model_performances:
  print(f'Model ({result["activation"]} Activation):')
  print(f'Accuracy: {result["accuracy"] * 100:.2f}%')
  print(f'F1 Score: {result["f1_score"]:.2f}')
  print(f'Precision: {result["precision"]:.2f}')
  print(f'Recall: {result["recall"]:.2f}')
  print(f'Duration: {(result["duration"]/60):.2f} Minutes')
  print('Confusion Matrix:')
  print(result['confusion_matrix'])
  print()
  print()

Model (Sigmoid() Activation):
Accuracy: 96.91%
F1 Score: 0.97
Precision: 0.97
Recall: 0.97
Duration: 2.17 Minutes
Confusion Matrix:
[[ 965    0    1    1    1    7    2    1    1    1]
 [   0 1128    2    1    0    1    2    0    1    0]
 [   0    2 1003   10    4    3    0    7    3    0]
 [   0    5    4  976    0   12    0    6    5    2]
 [   2    0    1    0  964    0    6    2    0    7]
 [   4    0    1    9    1  864    3    4    4    2]
 [   4    2    6    3    8   11  922    0    2    0]
 [   2   15   10    3    0    0    0  990    0    8]
 [   1    1    3    7    5   13    2    2  937    3]
 [   3    3    0    3   31   13    0   13    1  942]]


Model (Tanh() Activation):
Accuracy: 96.79%
F1 Score: 0.97
Precision: 0.97
Recall: 0.97
Duration: 2.17 Minutes
Confusion Matrix:
[[ 963    1    0    0    0    6    5    2    3    0]
 [   0 1123    1    1    0    1    1    0    7    1]
 [   7    2  991   13    1    1    2    8    7    0]
 [   1    0    2  986    0    5    0    3    3 

#Kesimpulan

Berdasarkan hasil run model neural network yang telah dibuat diatas menggunakan 3 buah aktivasi, dapat kita lihat bahwasanya tidak ada perbedaan yang signifikan dari tingkat akurasi, F1 Score, Precision, Recall, durasi, serta confusion matrix yang dihasilkan. Hal ini dapat dilihat melalui rincian dibawah.
1. Model Neural Network dengan fungsi aktivasi ReLU memiliki performa yang sedikit lebih baik dibandingkan 2 model lainnya, dimana ia memiliki akurasi sebesar 97.03%, durasi training selama2.16 Menit, serta F1 score, precision dan recall sebesar 0.97% **(Saat saya melakukan Training)**
2. Model Neural Network dengan fungsi aktivasi Sigmoid memiliki performa diperingkat kedua, dimana ia memiliki akurasi sebesar 96.91%, durasi training selama2.17 Menit, serta F1 score, precision dan recall sebesar 0.97 **(Saat saya melakukan Training)**
3. Model Neural Network dengan fungsi aktivasi Tanh memiliki performa diperingkat ketiga, dimana ia memiliki akurasi sebesar 96.79%, durasi training selama2.17 Menit, serta F1 score, precision dan recall sebesar 0.97 **(Saat saya melakukan Training)**